In [ ]:
# todo: check the model cfg: augmentations, policy

In [1]:
import torch
from torch import nn
from utils import parse_cfg
from darknet import Darknet

In [32]:
darknet = Darknet('cfg/yolov3.cfg')

to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
make_layers returns net_info as well. check whether it"s necessary


In [147]:
x = torch.rand((2, 171, 85))
print(x)
x.shape
centers_0 = x[..., 0]
print(centers_0)
l = torch.nn.Linear(85, 3)
x = l(x)
print(x)
print(centers_0)

tensor([[[0.5131, 0.9962, 0.9782,  ..., 0.4882, 0.8781, 0.0553],
         [0.7889, 0.1635, 0.6342,  ..., 0.5392, 0.3232, 0.2028],
         [0.8808, 0.8769, 0.6393,  ..., 0.7617, 0.9462, 0.5701],
         ...,
         [0.1751, 0.3089, 0.1370,  ..., 0.4392, 0.8342, 0.6726],
         [0.2612, 0.5479, 0.9476,  ..., 0.9596, 0.3426, 0.5875],
         [0.6884, 0.0758, 0.7555,  ..., 0.4939, 0.1039, 0.1235]],

        [[0.4519, 0.6910, 0.3362,  ..., 0.8940, 0.2874, 0.7648],
         [0.9725, 0.8989, 0.6647,  ..., 0.9592, 0.3737, 0.9424],
         [0.3361, 0.2852, 0.4986,  ..., 0.9100, 0.8995, 0.1346],
         ...,
         [0.9603, 0.0339, 0.8019,  ..., 0.0168, 0.1895, 0.1080],
         [0.0679, 0.2119, 0.1460,  ..., 0.6895, 0.4288, 0.3159],
         [0.0887, 0.6749, 0.4633,  ..., 0.9675, 0.6138, 0.2296]]])
tensor([[0.5131, 0.7889, 0.8808, 0.3287, 0.1931, 0.9697, 0.8799, 0.6192, 0.5272,
         0.3385, 0.4869, 0.1430, 0.8444, 0.7979, 0.2871, 0.9640, 0.7941, 0.0278,
         0.7052, 0.0793, 0

In [ ]:
torch.meshgrid()

In [194]:
import numpy as np

grid_size = 13
num_anchs = 3
# First, we need to add a center offset (cx, cy).
# 
grid = torch.arange(grid_size)
a, b = torch.meshgrid(grid, grid)
# 0, 0, ..., 12, 12
cx = a.type(torch.FloatTensor).view(-1, 1) # 0, 0, ..., 12, 12
cy = b.type(torch.FloatTensor).view(-1, 1) # 0, 1, ..., 11, 12
# 
x_y_offset = torch.cat((cx, cy), dim=1).repeat(1, num_anchs).view(-1, 2).unsqueeze(0)
print(torch.cat((cx, cy), 1).repeat(1, num_anchs))

# prediction[:,:,:2] += x_y_offset

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  1.],
        [ 0.,  2.,  0.,  2.,  0.,  2.],
        ...,
        [12., 10., 12., 10., 12., 10.],
        [12., 11., 12., 11., 12., 11.],
        [12., 12., 12., 12., 12., 12.]])


In [186]:
# grid_size = 13
# num_anchs = 3
# grid_x = torch.arange(grid_size).repeat(grid_size, 1).view([1, 1, grid_size, grid_size]).type(torch.FloatTensor)
# grid_y = torch.arange(grid_size).repeat(grid_size, 1).t().view([1, 1, grid_size, grid_size]).type(torch.FloatTensor)
# print(grid_x, grid_y)

tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
          [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.]]

In [26]:
# todo forward use both info and modules 
x = torch.randn((2, 3, 608, 608))

self_layer_list = darknet.layers_list
self_layer_info = darknet.layers_info[1:]

# cache the outputs for route and shortcut layers
outputs = []

for i, layer in enumerate(self_layer_list):
    
    name = self_layer_info[i]['name']
    # TODO: append something after each layer!!!!
    if name in ['convolutional', 'upsample']:
        x = layer(x)
        outputs.append(x)
        
    elif name == 'shortcut':
        # index which is used for shortcut (usually '-3')
        x = outputs[-1] + outputs[layer.frm]
#         x = x + outputs[layer.frm]
        outputs.append(None)
        
    elif name == 'route':
        to_cat = [outputs[route_idx] for route_idx in layer.routes]
        x = torch.cat(to_cat, dim=1)
        outputs.append(None)
        
    elif name == 'yolo':
        # input size: (B, (4+1+classes)*num_achors=255, Gi, Gi)
        B, C, w, h = x.size()
        # read layer's info
        anchors_list = layer.anchors
        classes = layer.classes
        in_width = layer.in_width
        num_anchs = len(anchors_list)
        # bbox coords + obj score + class scores
        num_feats = 4 + 1 + classes
        
        # transform the predictions
        # (B, (4+1+classes) * num_achors=255, Gi, Gi)
        # -> (B, Gi*Gi*num_anchors, (4+1+classes))
        x = x.view(B, num_anchs, num_feats, w, h)
        x = x.permute(0, 3, 4, 1, 2).contiguous() # (B, w, h, num_anchs, num_feats)
        x = x.view(1, h*w*num_anchs, num_feats)
        
        # normalize anchor coordinates to feet the output dimensions
        stride = in_width // w
        anchors_list = [(anchor[0] / stride, anchor[1] / stride) for anchor in anchors_list]
        
        # extract the output (tx, ty, tw, th) and transform it
        sig_txs = torch.sigmoid(x[:, :, 0])
        sig_tys = torch.sigmoid(x[:, :, 1])
        tws = x[:, :, 2]
        ths = x[:, :, 3]
        sig_objts = torch.sigmoid(x[:, :, 4])
        sig_clss = torch.sigmoid(x[:, :, 5:])
        
        
        
        try:
            pass
        
        except NameError:
            pass
        
        # output size: (B, \sum_i{Gi*Gi*num_anchors}, (4+1+classes))
        # todo append two times output and detections

SyntaxError: invalid syntax (<ipython-input-26-9ad423323e74>, line 23)

In [7]:
self_layer_list[0]

Sequential(
  (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_0): LeakyReLU(negative_slope=0.1)
)

In [120]:
# TODO proper transformation
a = torch.arange(1*(80+5)*3*5*5).view(1, 255, 5, 5)
# print(a)
print(a[:, :, 0, 1])
a = a.view(1, 3, 85, 5, 5).permute(0, 3, 4, 1, 2).contiguous().view(1, 75, 85)
print(a[:, 5, :])

tensor([[   1,   26,   51,   76,  101,  126,  151,  176,  201,  226,  251,  276,
          301,  326,  351,  376,  401,  426,  451,  476,  501,  526,  551,  576,
          601,  626,  651,  676,  701,  726,  751,  776,  801,  826,  851,  876,
          901,  926,  951,  976, 1001, 1026, 1051, 1076, 1101, 1126, 1151, 1176,
         1201, 1226, 1251, 1276, 1301, 1326, 1351, 1376, 1401, 1426, 1451, 1476,
         1501, 1526, 1551, 1576, 1601, 1626, 1651, 1676, 1701, 1726, 1751, 1776,
         1801, 1826, 1851, 1876, 1901, 1926, 1951, 1976, 2001, 2026, 2051, 2076,
         2101, 2126, 2151, 2176, 2201, 2226, 2251, 2276, 2301, 2326, 2351, 2376,
         2401, 2426, 2451, 2476, 2501, 2526, 2551, 2576, 2601, 2626, 2651, 2676,
         2701, 2726, 2751, 2776, 2801, 2826, 2851, 2876, 2901, 2926, 2951, 2976,
         3001, 3026, 3051, 3076, 3101, 3126, 3151, 3176, 3201, 3226, 3251, 3276,
         3301, 3326, 3351, 3376, 3401, 3426, 3451, 3476, 3501, 3526, 3551, 3576,
         3601, 3626, 3651, 3